## VGG16

In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
K.set_image_dim_ordering('th')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.25, random_state=32)

Using TensorFlow backend.


In [2]:
input = Input(shape=(3, 64, 64))
model = VGG16(weights=None, input_tensor=input, classes=3)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [3]:
datagen = ImageDataGenerator(rotation_range=0.9, zoom_range=0.5, shear_range=0.3, \
                             horizontal_flip=True, vertical_flip=True)
datagen.fit(train_data)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), steps_per_epoch=len(x_train)/100, \
                    epochs=1000, verbose=1, validation_data=(x_val_train, y_val_train))

Epoch 1/1000
11/12 [==========================>...] - ETA: 1s - loss: 1.0924 - acc: 0.4303

KeyboardInterrupt: 

In [ ]:
test_data = np.load('test.npy')
test_id = np.load('test_id.npy')

pred = model.predict(test_data)
df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)